In [ ]:
from fmr_scripts import FMR #if running in another directory - requires PYTHONPATH or sys.path to contain directory containing fmr_scripts
#import FMR #if running in same directory
import numpy as np
import matplotlib.pyplot as plt
import glob,os
from scipy.interpolate import interp1d

#### Read in data

In [ ]:
#directory = '/Volumes/Transcend/Suzuki/Data/ppblsmo/FMR/ppblsmo18A-IP110/'
#olddir = os.getcwd()
#os.chdir(directory)
filenames = glob.glob('*.csv')
for f in filenames:
    print f

In [ ]:
#sample specific
toremove = ['ppblsmo18A-IP110-20180702-IP-19000MHz-000.0deg-+3dBm3.0OeRMS-0-21.csv']
for t in toremove:
    filenames.remove(t)
for f in filenames:
    print f

In [ ]:
Hmeass = []
dPdHs = []
phases = []
freqs = []
angs = []
Hreadscales = []
Hreadoffs = []
for filename in filenames:
    print filename
    Hmeas, dPdH, phase = FMR.readFileTruncatePos(filename)
    Hmeass.append(H)
    dPdHs.append(dPdH)
    phases.append(phase)
    freq = int(filename.partition('MHz')[0].rpartition('-')[-1])/1000.
    freqs.append(freq)
    ang = float(filename.partition('deg')[0].rpartition('-')[-1])
    angs.append(ang)
    Hreadscale = float(filename.partition('scale')[-1].partition('-')[0])
    Hreadscales.append(Hreadscale)
    Hreadoffpart = filename.partition('off')[-1].partition('-')
    Hreadoff = float(Hreadoffpart[0]) if Hreadoffpart[0] != '' else -float(Hreadoffpart[-1].partition('-')[0])
    Hreadoffs.append(Hreadoff)

#os.chdir(olddir)

#### Gaussmeter calibration

In [ ]:
gaussmeter = np.load('../0 H-field Calibration/190626/fieldcalib190626.npz')
print gaussmeter['h_read_scale']
print Hreadscales
print gaussmeter['h_read_offset']
print Hreadoffs

In [ ]:
gaussfit = interp1d(gaussmeter['twosupply_measuredH'], gaussmeter['twosupply_actualH'],kind='cubic')

Hs = []
for i,Hmeas in enumerate(Hmeass):
    #field recorded in the file is field_measured*h_read_scale+h_read_off - instead we are correcting this to a better calibration from measured -> actual H
    Hs.append(gaussfit((np.array(Hmeas)-Hreadoffs[i])/Hreadscales[i]))

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print filenames[i]
    print str(i)+': '+str(freqs[i])+' GHz, '+str(angs[i])+' deg'
    plt.plot(H,dPdHs[i],label='corrected')
    plt.plot(Hmeass[i],dPdHs[i],label='arbitrary scaling')
    imax = np.argmax(dPdHs[i])
    plt.plot(H[imax],dPdHs[i][imax],'*')
    imin = np.argmin(dPdHs[i])
    plt.plot(H[imin],dPdHs[i][imin],'*')
    plt.legend()
    plt.show()

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print freqs[i]
    plt.plot(H,phases[i])
    plt.show()

#### Subtract background

In [ ]:
%matplotlib inline
dPdHoffsets = []
windows = [10]*len(dPdHs)
for i,H in enumerate(Hs):
    dPdH = dPdHs[i]
    fitBG = FMR.subtractBG(H,dPdH,window=windows[i])
    print str(i)+': '+str(freqs[i])+' GHz, '+str(angs[i])+' deg'
    plt.show()
    dPdHoffset = dPdH - fitBG
    dPdHoffsets.append(dPdHoffset)

In [ ]:
#i = 2
#H = Hs[i]
#dPdH = dPdHs[i]
#fitBG = FMR.subtractBG(Hs[i],dPdH,window=10)
#plt.show()
#print freqs[i]

#### Fit peaks

In [ ]:
NPeaks = [2]*len(freqs)
guessPos = ['Center']*len(freqs)
widths=[3]*len(freqs)
fits = []
%matplotlib inline
for i,H in enumerate(Hs):
    dPdHoffset = dPdHoffsets[i]
    guess = FMR.guessN(H,dPdHoffset,NPeaks[i],width=widths[i],pos=guessPos[i])
    fit = FMR.fitFMR(H,dPdHoffset,guess)
    fits.append(fit)
    print str(i)+': '+str(freqs[i])+' GHz, '+str(angs[i])+' deg'
    plt.show()

In [ ]:
#%matplotlib inline
#i = 12
#print guessPos[i]
#print NPeaks[i]
#print widths[i]
#print str(i)+': '+str(freqs[i])
#dPdHoffset = dPdHoffsets[i]
#guess = FMR.guessN(Hs[i],dPdHoffset,1,width=3,pos='Center')
#fit = FMR.fitFMR(Hs[i],dPdHoffset,guess,debug=True)
#plt.show()
#guess = FMR.guessN(Hs[i],dPdHoffset,2,width=3,pos='Left')
#fit = FMR.fitFMR(Hs[i],dPdHoffset,guess,debug=True)
#plt.show()
#print fit[0]

#### Save curves and fits

In [ ]:
fname = filenames[0].partition('-Hread')[0]
outfile = fname+'-curves'
csort = np.lexsort([freqs,angs])
np.savez(outfile, frequenciesGHz = np.array(freqs)[csort], anglesdeg = np.array(angs)[csort],
         fieldOe = np.array(Hs)[csort], powerderivV = np.array(dPdHoffsets)[csort], phaseDeg = np.array(phases)[csort],
         fittedpeaks = np.array(fits)[csort])

#### Extract fit parameters

In [ ]:
HFMR = []
errHFMR = []
deltaH = []
errdeltaH = []
phaseLorentzian = []
errphaseLorentzian = []
peakMagnitude = []
errpeakMagnitude = []
for i,f in enumerate(fits):
    peakMagnitude.append(np.array([f[0][0][0+n*4] for n in NPeaks[i]]))
    HFMR.append(np.array([f[0][0][1+n*4] for n in NPeaks[i]]))
    phaseLorentzian.append(np.array([f[0][0][2+n*4] for n in NPeaks[i]]))
    deltaH.append(np.array([f[0][0][3+n*4] for n in NPeaks[i]]))
    errpeakMagnitude.append(np.array([f[1][0][0+n*4] for n in NPeaks[i]]))
    errHFMR.append(np.array([f[1][0][1+n*4] for n in NPeaks[i]]))
    errphaseLorentzian.append(np.array([f[1][0][2+n*4] for n in NPeaks[i]]))
    errdeltaH.append(np.array([f[1][0][3+n*4] for n in NPeaks[i]]))

In [ ]:
#essentially a reslicing
N = np.max(NPeaks)
freqsN = []
angsN = []
HFMRN = []
errHFMRN = []
deltaHN = []
errdeltaHN = []
phaseLorentzianN = []
errphaseLorentzianN = []
peakMagnitudeN = []
errpeakMagnitudeN = []
for n in np.arange(0,N):
    whichcurves = np.where(N > n)[0]
    freqsN.append(freqs[whichcurves])
    angsN.append(angs[whichcurves])
    HFMRN.append(np.array([HFMR[i][n] for i in whichcurves]))
    errHFMRN.append(np.array([errHFMR[i][n] for i in whichcurves]))
    deltaHN.append(np.array([deltaH[i][n] for i in whichcurves]))
    errdeltaHN.append(np.array([errdeltaH[i][n] for i in whichcurves]))
    phaseLorentzianN.append(np.array([phaseLorentzian[i][n] for i in whichcurves]))
    errphaseLorentzianN.append(np.array([errphaseLorentzian[i][n] for i in whichcurves]))
    peakMagnitudeN.append(np.array([peakMagnitude[i][n] for i in whichcurves]))
    errpeakMagnitudeN.append(np.array([errpeakMagnitude[i][n] for i in whichcurves]))

In [ ]:
#HFMRfit = np.polyfit(HFMR[0][10:],freqs[10:],1)
#print HFMRfit
#Meff = -HFMRfit[1]/HFMRfit[0]
#print Meff
#gop = HFMRfit[0]*h/(muB*mu0) #insert h, muB, mu0 in correct units

In [ ]:
%matplotlib inline
for n in np.arange(0,N):
    plt.errorbar(freqsN[n],HFMRN[n],fmt='.',yerr=errHFMRN[n])
#    plt.plot(np.poly1d(HFMRfit)(HFMRN[n]),HFMRN[n],'-')
plt.ylabel('HFMR (Oe)')
plt.xlabel('Frequency (Hz)')

In [ ]:
%matplotlib inline
for n in np.arange(0,N):
    plt.errorbar(freqsN[n],deltaHN[n],fmt='.',yerr=errdeltaHN[n])
plt.ylabel('FMR Linewidth (Oe)')
plt.xlabel('Frequency (Hz)')

In [ ]:
%matplotlib inline
for n in np.arange(0,N):
    plt.errorbar(freqsN[n],phaseLorentzianN[n],fmt='.',yerr=errphaseLorentzianN[n])
plt.ylabel('Phase (deg)')
plt.xlabel('Frequency (Hz)')

In [ ]:
%matplotlib inline
for n in np.arange(0,N):
    plt.errorbar(freqsN[n],peakMagnitudeN[n],fmt='.',yerr=errpeakMagnitudeN[n])
plt.ylabel('Magnitude (a.u.)')
plt.xlabel('Frequency (Hz)')

#### Save parameters

In [ ]:
np.savez(fname, frequenciesGHz = freqs, anglesdeg = angs,
         resonantFieldsOe = HFMR, linewidthsOe = deltaH,
         resonantFieldsErrorOe = errHFMR, linewidthsErrorOe = errdeltaH,
         peakPhasesdeg = phaseLorentzian, peakMagnitude = peakMagnitude,
         peakPhasesErrordeg = errphaseLorentzian, peakMagnitudeError = errpeakMagnitude)